<a href="https://colab.research.google.com/github/GianEscher/quantum-lab-finance/blob/main/QAOA_estudo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
!pip install qiskit
!pip install qiskit-finance

primeiro teste de código - recuperar dados pelo yahoo finance

erro: 1 Failed download:
- FB: No timezone found, symbol may be delisted
---------------------------------------------------------------------------
DOcplexException                          Traceback (most recent call last)
<ipython-input-67-81e21955475c> in <cell line: 68>()
     66 
     67 portfolio = PortfolioOptimization(expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget)
---> 68 qp = portfolio.to_quadratic_program()
     69 
     70 # Crie uma instância do algoritmo QAOA

7 frames
/usr/local/lib/python3.10/dist-packages/docplex/mp/error_handler.py in docplex_fatal(msg, *args)
    254     resolved_message = resolve_pattern(msg, args)
    255     docplex_error_stop_here()
    
--> 256     raise DOcplexException(resolved_message)
    257 
    258 def is_docplex_debug():

:**DOcplexException: Expecting valid float number, got: nan**


In [101]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
#from qiskit.finance.applications.ising import portfolio
from qiskit_finance.applications.optimization import PortfolioOptimization

from qiskit import Aer
#from qiskit.aqua import aqua_globals, QuantumInstance
#from qiskit.aqua.algorithms import QAOA
from qiskit.algorithms.minimum_eigensolvers import QAOA
#from qiskit.aqua.components.optimizers import COBYLA
from qiskit.algorithms.optimizers import COBYLA

yf.pdr_override()

# Defina os ativos para os quais queremos os dados!!!!!!!!!!!!!
assets = ['AAPL', 'MSFT', 'GOOG', 'FB', 'AMZN']

# Defina o intervalo de datas para o qual queremos os dados
start_date = '2020-01-01'
end_date = '2021-12-31'

# Obtenha os dados do Yahoo Finance!!!!!!!!!!!
data = pdr.get_data_yahoo(assets, start=start_date, end=end_date)

# Use apenas o preço de fechamento!!!!!!!!
data = data['Close']

# Calcule os retornos logarítmicos, que são mais apropriados para a análise financeira!!!!!!
returns = np.log(data / data.shift(1))

# Calcule os retornos médios e a matriz de covariância!!!!!
mu = returns.mean().values

# Generate expected return and covariance matrix from (random) time-series
sigma = returns.cov().values

# Defina o orçamento
budget = 2

# Crie uma instância do problema de otimização de portfólio
#portfolio = portfolio.Portfolio(expected_returns=mu, covariances=sigma, budget=budget) PROVAVELMENTE DEPRECIADO
# AO INVES DE --- from qiskit.finance.applications.ising import portfolio
#USAR ----------- from qiskit_finance.applications.optimization import PortfolioOptimization

#NOVA CRIAÇÃO DE OBJETO portfolio REQUER FATOR DE RISCO
q = float(0.5)

portfolio = PortfolioOptimization(expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget)

print("RETORNOS")
print(returns)

print("PORTFOLIO")
print(portfolio)

# Crie uma instância do algoritmo QAOA
#qaoa = QAOA(optimizer=COBYLA(), quantum_instance=QuantumInstance(backend=Aer.get_backend('statevector_simulator')))
#SUPOSTAMENTE DEPRECIADA - LER O BLOCO DE TEXO ABAIXO!

# Execute o algoritmo QAOA para encontrar a solução ótima
#result = qaoa.compute_minimum_eigenvalue(qubit_op)

# Print the result
#print(result)

[*********************100%***********************]  5 of 5 completed

1 Failed download:
- FB: No timezone found, symbol may be delisted
RETORNOS
                AAPL      AMZN  FB      GOOG      MSFT
Date                                                  
2020-01-02       NaN       NaN NaN       NaN       NaN
2020-01-03 -0.009770 -0.012213 NaN -0.004919 -0.012530
2020-01-06  0.007937  0.014776 NaN  0.024358  0.002581
2020-01-07 -0.004714  0.002089 NaN -0.000624 -0.009160
2020-01-08  0.015958 -0.007839 NaN  0.007850  0.015803
...              ...       ...  ..       ...       ...
2021-12-23  0.003637  0.000184 NaN  0.001316  0.004462
2021-12-27  0.022715 -0.008212 NaN  0.006243  0.022921
2021-12-28 -0.005784  0.005827 NaN -0.010974 -0.003510
2021-12-29  0.000502 -0.008592 NaN  0.000386  0.002049
2021-12-30 -0.006600 -0.003294 NaN -0.003432 -0.007721

[504 rows x 5 columns]
PORTFOLIO


https://github.com/qiskit-community/qiskit-aqua/issues/504  ----> essa página permite entender que a seguinte instância do QAOA

> qaoa = QAOA(optimizer=COBYLA(), quantum_instance=QuantumInstance(backend=Aer.get_backend('statevector_simulator')))

 -> aparenta depender do framework aqua, e portanto estaria depreciada

> DOCUMENTAÇÃO ORIGINAL REFERENTE À QAOA(optmizer, quantum_intance, backend) e quantum_instance = QuantumInstance(backend=backend, seed=seed, seed_transpiler=seed) REMOVIDA



In [123]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf

#yf.pdr_override()

#from qiskit.finance.applications.ising import portfolio
from qiskit_finance import *
from qiskit_finance.applications import *
from qiskit_finance.applications.optimization import PortfolioOptimization

from qiskit import Aer
#from qiskit.aqua import aqua_globals, QuantumInstance
#from qiskit.aqua.algorithms import QAOA
from qiskit.algorithms.minimum_eigensolvers import QAOA
#from qiskit.aqua.components.optimizers import COBYLA
from qiskit.algorithms.optimizers import COBYLA

from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import numpy as np
import matplotlib.pyplot as plt
import datetime


yf.pdr_override()

# Defina os ativos para os quais queremos os dados!!!!!!!!!!!!!
assets = ['AAPL', 'MSFT', 'META', 'GOOG', 'AMZN']#ChatGpt utilizou FB ao invés de META - gerando séries de 'nan'

# Defina o intervalo de datas para o qual queremos os dados
start_date = '2022-12-01'
end_date = '2022-12-31'

# Obtenha os dados do Yahoo Finance!!!!!!!!!!!
#data_raw = pdr.get_data_yahoo(assets, start=start_date, end=end_date) 
data_raw = pdr.DataReader(assets, start=start_date, end=end_date)

# Use apenas o preço de fechamento!!!!!!!!
data = []
data = data_raw['Close']

#CALCULAR OS RETORNOS LOGARITMICOS GERA ERRO NA OTIMIZAÇÃO POR QAOA
#o laço while abaixo calcula os erros percentuais consecutivos

returns_dictio = {}
i = 0
while(i<len(assets)):
  j=1
  stocks_registry = []
  key = assets[i]
  while(j<len(data[key])):
    stocks_registry.append((data[key][j]-data[key][j-1])/data[key][j-1])
    j+=1
  returns_dictio[key] = stocks_registry
  i +=1

returns = pd.DataFrame(returns_dictio)
print(returns)

# Calcule os retornos médios e a matriz de covariância!!!!!
mu = returns.mean().values
sigma = returns.cov().values

print(mu)
print(sigma)

# Defina o orçamento
budget = 2

# Crie uma instância do problema de otimização de portfólio
#portfolio = portfolio.Portfolio(expected_returns=mu, covariances=sigma, budget=budget)

q = float(0.5)

portfolio = PortfolioOptimization(expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget)
qp = portfolio.to_quadratic_program()

# Crie uma instância do algoritmo QAOA
#qaoa = QAOA(optimizer=COBYLA(), quantum_instance=QuantumInstance(backend=Aer.get_backend('statevector_simulator')))

# Execute o algoritmo QAOA para encontrar a solução ótima
#result = qaoa.compute_minimum_eigenvalue(qubit_op)

# Print the result
#print(result)

[*********************100%***********************]  5 of 5 completed
        AAPL      MSFT      META      GOOG      AMZN
0  -0.003371  0.001296  0.025324 -0.004443 -0.014346
1  -0.007983 -0.018901 -0.008584 -0.009521 -0.033146
2  -0.025370 -0.020304 -0.067876 -0.025633 -0.030326
3  -0.013785 -0.003060 -0.001665 -0.022197  0.002380
4   0.012133  0.012399  0.012288 -0.012612  0.021366
5  -0.003435 -0.008003  0.004942 -0.009367 -0.013946
6   0.016390  0.028889 -0.010267  0.005265  0.016388
7   0.006782  0.017465  0.047424  0.024476  0.021425
8  -0.015536  0.001168  0.011985 -0.005634 -0.009839
9  -0.046854 -0.031918 -0.044740 -0.043122 -0.034178
10 -0.014579 -0.017349  0.028239 -0.003728 -0.006670
11 -0.015910 -0.017328 -0.041447 -0.018820 -0.033462
12 -0.000529  0.005614  0.022799  0.005384  0.003180
13  0.023809  0.010877  0.022803  0.006917  0.018547
14 -0.023773 -0.025529 -0.022044 -0.022050 -0.034344
15 -0.002798  0.002267  0.007855  0.017562  0.017425
16 -0.013878 -0.007414 -0.0098


ABAIXO SEGUE ERRO GERADO PELO QAOA, SE CÁLCULO DO RETORNO LOGARITMICO FOR USADO:

> ValueError                                Traceback (most recent call last)
<ipython-input-95-0fb84eb29595> in <cell line: 7>()
      5 qaoa_mes = QAOA(sampler=Sampler(), optimizer=cobyla, reps=3)
      6 qaoa = MinimumEigenOptimizer(qaoa_mes)
----> 7 result = qaoa.solve(qp)
      8 
      9 print_result(result)
      


> 11 frames
/usr/local/lib/python3.10/dist-packages/qiskit/circuit/library/pauli_evolution.py in _to_sparse_pauli_op(operator)
    171 
    172     if any(np.iscomplex(sparse_pauli.coeffs)):
--> 173         raise ValueError("Operator contains complex coefficients, which are not supported.")
    174     if any(isinstance(coeff, ParameterExpression) for coeff in sparse_pauli.coeffs):
    175         raise ValueError("Operator contains ParameterExpression, which are not supported.")

ValueError: Operator contains complex coefficients, which are not supported.








In [124]:
algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
qaoa_mes = QAOA(sampler=Sampler(), optimizer=cobyla, reps=3)
qaoa = MinimumEigenOptimizer(qaoa_mes)
result = qaoa.solve(qp)

print_result(result)

Optimal: selection [0. 1. 1. 0. 0.], value 0.0034

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[1 0 0 0 1]	0.0133		0.0752
[0 1 1 0 0]	0.0034		0.0674
[0 0 1 0 1]	0.0069		0.0654
[0 1 0 1 0]	0.0098		0.0645
[0 0 0 1 1]	0.0133		0.0605
[1 1 0 0 0]	0.0099		0.0576
[1 0 1 0 0]	0.0070		0.0566
[0 0 1 1 0]	0.0071		0.0557
[0 1 0 0 1]	0.0097		0.0547
[1 0 0 1 0]	0.0135		0.0518
[0 1 1 0 1]	0.0106		0.0400
[1 0 1 1 0]	0.0143		0.0342
[1 0 0 0 0]	0.0066		0.0322
[0 1 0 1 1]	0.0169		0.0303
[1 1 1 0 0]	0.0106		0.0303
[0 0 1 1 1]	0.0142		0.0293
[1 0 0 1 1]	0.0205		0.0244
[0 1 0 0 0]	0.0030		0.0234
[1 1 0 1 0]	0.0170		0.0215
[1 1 0 0 1]	0.0169		0.0215
[0 0 1 0 0]	0.0001		0.0205
[0 1 1 1 0]	0.0106		0.0205
[1 0 1 0 1]	0.0142		0.0205
[0 0 0 1 0]	0.0066		0.0166
[0 0 0 0 1]	0.0064		0.0127
[1 1 1 1 1]	0.0259		0.0088
[0 0 0 0 0]	0.0000		0.0020
[1 0 1 1 1]	0.0218		0.0010
[1 1 1 1 0]	0.0181		0.0010


ABAIXO SEGUE IMPLEMENTAÇÃO ATUAL DO QAOA, UTILIZANDO VALORES ALEATÓRIOS GERADOS POR RandomDataProvider()



> DISPONÍVEL EM: https://qiskit.org/ecosystem/finance/tutorials/01_portfolio_optimization.html



In [87]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import numpy as np
import matplotlib.pyplot as plt
import datetime

# set number of assets (= number of qubits)
num_assets = 4
seed = 123

# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider(
    tickers=stocks,
    start=datetime.datetime(2016, 1, 1),
    end=datetime.datetime(2016, 1, 30),
    seed=seed,
)
data.run()
mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()



In [88]:
q = 0.5  # set risk factor
budget = num_assets // 2  # set budget
penalty = num_assets  # set parameter to scale the budget penalty term

portfolio = PortfolioOptimization(
    expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget
)
qp = portfolio.to_quadratic_program()
qp

<QuadraticProgram: minimize 0.001270694296030004*x_0^2 + 7.34022166934733e-05*..., 4 variables, 1 constraints, 'Portfolio optimization'>

In [89]:
def print_result(result):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = portfolio.to_quadratic_program().objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))

In [ ]:
algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
qaoa_mes = QAOA(sampler=Sampler(), optimizer=cobyla, reps=3)
qaoa = MinimumEigenOptimizer(qaoa_mes)
result = qaoa.solve(qp)

print_result(result)